In [ ]:
# Import necessary libraries
from typing import Dict, List, Any
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.tools import BaseTool
from langchain_core.tools import Tool

# Updated import for memory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
import re

In [ ]:
llm = ChatOllama(model="llama3.2:1b", temperature=0)

In [ ]:
class QuestionGeneratorInput(BaseModel):
    language: str = Field(description="Programming language for the question")
    field: str = Field(description="Technical field for the question")
    dsa: str = Field(description="Data structure or algorithm focus")
    difficulty: str = Field(description="Difficulty level: easy, medium, or hard")

class EvaluationInput(BaseModel):
    answer: str = Field(description="User's answer to evaluate")
    question: str = Field(description="The question that was asked")
    difficulty: str = Field(description="Difficulty level of the question")

class QuestionSelectorInput(BaseModel):
    current_difficulty: str = Field(description="Current difficulty level")
    eval_score: int = Field(description="Current evaluation score")
    question_count: int = Field(description="Number of questions asked so far")

class ReportGeneratorInput(BaseModel):
    answers: List[str] = Field(description="List of all user answers")
    eval_scores: List[int] = Field(description="List of evaluation scores")
    difficulty_counts: Dict[str, int] = Field(description="Count of questions by difficulty")

In [ ]:
def question_generator(input_data: QuestionGeneratorInput) -> str:
    """Generate a question based on language, field, DSA concept and difficulty level."""
    prompt = f"You are a teacher with expertise in {input_data.language}, {input_data.field}, and {input_data.dsa}. Ask a {input_data.difficulty} level question."
    return llm.invoke(prompt).content

In [ ]:
def evaluate_answer(input_data: EvaluationInput) -> Dict[str, Any]:
	"""you are a highly educated expert and your job is to perform strict evaluation of the answer based on the given question"""
	# Simple evaluation logic - can be made more sophisticated
	prompt = f"""you are a highly educated expert and your job is to perform strict evaluation of the answer based on the given question"{input_data.question}" 
	And the answer: "{input_data.answer}"
	Evaluate this {input_data.difficulty} level answer on a scale of 0-10.
	make sure to respond with a number between 0 and 10."""
	score = llm.invoke(prompt)
	try:
		match = re.search(r'\d+', score.content)
		if match:
				return int(match.group())
	except:
		return 5






In [ ]:
def choose_next_question(input_data: QuestionSelectorInput) -> Dict[str, Any]:
    """Choose the next question difficulty based on performance."""
    curr_type = input_data.current_difficulty
    eval_score = input_data.eval_score
    question_count = input_data.question_count
    
    if curr_type == "easy" and eval_score >= 5:
        new_type = "medium"
    elif curr_type == "medium":
        if eval_score >= 8:
            new_type = "hard"
        elif eval_score < 5:
            new_type = "easy"
        else:
            new_type = "medium"
    elif curr_type == "hard" and eval_score < 7:
        new_type = "medium"
    else:
        new_type = curr_type

    should_continue = question_count < 6
    
    return {
        "new_difficulty": new_type,
        "should_continue": should_continue
    }

In [ ]:
def generate_final_report(input_data: ReportGeneratorInput) -> str:
    """Generate a final report of the user's performance."""
    prompt = f"""Based on these answers and scores, write a short evaluation highlighting strengths, weaknesses, and areas for improvement:
    - Number of easy questions answered: {input_data.difficulty_counts.get('easy', 0)}
    - Number of medium questions answered: {input_data.difficulty_counts.get('medium', 0)}
    - Number of hard questions answered: {input_data.difficulty_counts.get('hard', 0)}
    - Average score: {sum(input_data.eval_scores) / len(input_data.eval_scores) if input_data.eval_scores else 0}
    """
    return llm.invoke(prompt).content

In [ ]:
system_prompt = """You are an adaptive quiz agent. Your job is to:
1. Ask questions about programming topics based on language, field, and data structures/algorithms
2. Evaluate user answers
3. Adjust difficulty based on performance
4. Generate a final report after 15 questions

Follow this process:
1. Start with an easy question about the specified language, field, and DSA concept
2. After receiving an answer, evaluate it
3. Determine the next question's difficulty based on performance
4. After 15 questions, generate a final report

Track the following:
- Current difficulty level (starts as "easy")
- Total evaluation score
- Question count
- Number of easy/medium/hard questions answered
- List of all answers and their evaluation scores

When generating the final report, analyze the user's performance and provide specific feedback.
You have to ask the questions and evaluate answers
"""

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START, END
from langgraph.graph.message import add_messages


class chatstate(TypedDict):
	messages: Annotated[list, add_messages]


graph = StateGraph(chatstate)


In [ ]:
def end_quiz(i):
    if i<15:
        return "next question"
    else:
        return "create report"
    

In [ ]:
graph.add_node("question_generator",question_generator)
graph.add_node("evaluate_answer",evaluate_answer)
graph.add_node("choose_next_question",choose_next_question)
graph.add_node("generate_final_report",generate_final_report)



In [ ]:

graph.add_edge(START,"question_generator")
graph.add_edge("question_generator","evaluate_answer")
graph.add_edge("evaluate_answer","choose_next_question")
# graph.add_edge("choose_next_question","question_generator")


In [ ]:
graph.add_conditional_edges("choose_next_question",
end_quiz,{
"next question": "question_generator",
"create report" : "generate_final_report"
}
)

In [ ]:
builder = graph.compile()

In [ ]:
from IPython.display import Image, display

display(Image(builder.get_graph().draw_mermaid_png()))

In [ ]:
builder.invoke()

In [ ]:
# Modified version of your code with fixes for the graph invocation

# First, modify your existing functions to work with the message state:
def question_generator_node(state):
    # Extract necessary information from state or use defaults
    language = "Python"  # Default or extract from state
    field = "Software Engineering"  # Default or extract from state
    dsa = "Algorithm Design"  # Default or extract from state  
    difficulty = "easy"  # Start with easy
    
    # Create input data
    input_data = QuestionGeneratorInput(
        language=language,
        field=field,
        dsa=dsa,
        difficulty=difficulty
    )
    
    # Call the actual function
    question = question_generator(input_data)
    
    # Add the generated question to the messages
    state["messages"].append({"role": "assistant", "content": question})
    state["question"] = question
    state["difficulty"] = difficulty
    state["question_count"] = 1
    
    return state

def evaluate_answer_node(state):
    # Extract the answer from user input
    user_messages = [m for m in state["messages"] if m.get("role") == "user"]
    if not user_messages:
        # No user response yet
        return state
    
    answer = user_messages[-1]["content"]
    question = state.get("question", "")
    difficulty = state.get("difficulty", "easy")
    
    # Create input data
    input_data = EvaluationInput(
        answer=answer,
        question=question,
        difficulty=difficulty
    )
    
    # Get the evaluation score
    result = evaluate_answer(input_data)
    score = result  # Assuming your evaluate_answer returns a score
    
    # Add to state
    if "eval_scores" not in state:
        state["eval_scores"] = []
    state["eval_scores"].append(score)
    state["current_score"] = score
    
    # Add evaluation message
    state["messages"].append({"role": "assistant", "content": f"Your answer scored {score}/10"})
    
    return state

def choose_next_question_node(state):
    # Get current state
    current_difficulty = state.get("difficulty", "easy")
    eval_score = state.get("current_score", 5)
    question_count = state.get("question_count", 1)
    
    # Create input data
    input_data = QuestionSelectorInput(
        current_difficulty=current_difficulty,
        eval_score=eval_score,
        question_count=question_count
    )
    
    # Choose next question
    result = choose_next_question(input_data)
    new_difficulty = result["new_difficulty"]
    should_continue = result["should_continue"]
    
    # Update state
    state["difficulty"] = new_difficulty
    state["should_continue"] = should_continue
    state["question_count"] = question_count + 1
    
    # Track difficulty counts
    if "difficulty_counts" not in state:
        state["difficulty_counts"] = {"easy": 0, "medium": 0, "hard": 0}
    state["difficulty_counts"][current_difficulty] = state["difficulty_counts"].get(current_difficulty, 0) + 1
    
    return state

def generate_final_report_node(state):
    # Get data from state
    eval_scores = state.get("eval_scores", [])
    difficulty_counts = state.get("difficulty_counts", {"easy": 0, "medium": 0, "hard": 0})
    
    # Extract all user answers
    answers = [m["content"] for m in state["messages"] if m.get("role") == "user"]
    
    # Create input data
    input_data = ReportGeneratorInput(
        answers=answers,
        eval_scores=eval_scores,
        difficulty_counts=difficulty_counts
    )
    
    # Generate the report
    report = generate_final_report(input_data)
    
    # Add to state
    state["messages"].append({"role": "assistant", "content": f"## Final Report\n\n{report}"})
    state["final_report"] = report
    
    return state

def end_quiz_condition(state):
    question_count = state.get("question_count", 0)
    if question_count < 15:
        return "next_question"
    else:
        return "create_report"

# Recreate the graph with the modified nodes
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class chatstate(TypedDict):
    messages: Annotated[list, add_messages]
    question: str
    difficulty: str
    question_count: int
    current_score: int
    eval_scores: list
    difficulty_counts: dict
    should_continue: bool
    final_report: str

# Create the graph
graph = StateGraph(chatstate)

# Add nodes
graph.add_node("question_generator", question_generator_node)
graph.add_node("evaluate_answer", evaluate_answer_node)
graph.add_node("choose_next_question", choose_next_question_node)
graph.add_node("generate_final_report", generate_final_report_node)

# Add edges
graph.add_edge(START, "question_generator")
graph.add_edge("question_generator", "evaluate_answer")
graph.add_edge("evaluate_answer", "choose_next_question")
graph.add_conditional_edges(
    "choose_next_question",
    end_quiz_condition,
    {
        "next_question": "question_generator",
        "create_report": "generate_final_report"
    }
)
graph.add_edge("generate_final_report", END)

# Compile the graph
builder = graph.compile()

# Now invoke the graph with proper initial state
initial_state = {
    "messages": [
        {"role": "system", "content": "You are an adaptive quiz agent that generates programming questions."},
        {"role": "user", "content": "I want to practice Python programming questions about data structures."}
    ]
}

# The correct way to invoke the graph:
# result = builder.invoke(initial_state)
print("Now you can run: result = builder.invoke(initial_state)")

In [ ]:
result = builder.invoke(initial_state)
